In [5]:
import sys

IN_COLAB = 'google.colab' in sys.modules

if not IN_COLAB:

    from git import Repo

    # Initialize the Git repository object
    repo = Repo(".", search_parent_directories=True)

    # Get the root directory of the Git project
    root_dir = repo.git.rev_parse("--show-toplevel")

    from pathlib import Path

    # Set up path for custom importer modules
    importer_module = root_dir + '/dataloader/'
    sys.path.insert(0, importer_module)

    # Insert here your local path to the dataset
    data_path = input("Path to the dataset: ")

else:

    from google.colab import drive
    drive.mount('/content/drive')

    # On Colab the path to the module ti fixed once you have
    # corretly set up the project with gitsetup.ipynb
    fixed_path = '/content/drive/MyDrive/Github/visiope/dataloader/'
    sys.path.insert(0, fixed_path)

    # Insert here the path to the dataset on your drive
    data_path = input("Path to the dataset: ")

In [ ]:
import numpy as np
import os
from loader import Ai4MarsImporter, Ai4MarsProcessor, Ai4MarsData

print("Making the npdataset")
data_import = Ai4MarsImporter()
X, y = data_import(path=data_path, num_of_images=1000)

# set up dataset directory
save_path = input("Path to save the npdataset: ")

if not os.path.exists(save_path) : 
    os.makedirs(save_path)

save_path_X = save_path + 'X'
np.save(save_path_X, X)

save_path_y = save_path + 'y'
np.save(save_path_y, y)